Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from pickle import dump, load
import os
from yahooquery import Ticker
from tqdm.notebook import tqdm
import timeit

1) Download and extract https://www.kaggle.com/datasets/footballjoe789/us-stock-dataset

2) Copy Stocks/ to the same directory as this notebook. You can delete the other files as they are not used (yet, at least).

In [61]:
class stock_tickers():
    """A work-in-progress. Goal is to take stock ticker symbols and return a list of search terms for NLP web scraping. Officers, affiliated companies, company name, etc."""
    def __init__(self, file_path = 'Stocks/', file_ext = '.csv'):
        self.delete_small_files(file_path, file_ext, min_size = 1000)
        self.load_symbols(file_path, file_ext)
        self.ticker_list_to_dataframe()
        self.get_company_list()
        self.process_information()

    def load_symbols(self, file_path = 'Stocks/', file_ext = '.csv'):
        """Find all the stock symbols in the directory and return a list of them."""
        self.stocks_symbols = [name.split('.')[0].split("""\\""")[-1] for name in glob (file_path+'*'+file_ext)]
        return self.stocks_symbols

    def delete_small_files(self, file_path = 'Stocks/', file_ext = '.csv', min_size = 1000):
        """Delete any files smaller than min_size bytes."""
        self.load_symbols()
        file_sizes = [os.path.getsize(file_path+name+file_ext) for name in self.stocks_symbols]
        if min(file_sizes) < min_size:
            for i in range(len(file_sizes)):
                if file_sizes[i] < min_size:
                    os.remove(file_path+self.stocks_symbols[i]+file_ext)

    def ticker_list_to_dataframe(self):
        self.data = pd.DataFrame(self.stocks_symbols, columns = ['Ticker'])

    def get_company(self, ticker):
        """Return the company name for a given ticker."""
        #return self.yh_tickers[ticker].info['longName']
        pass

    def get_company_officers(self, index):
        try:
            officers = tickers.yh_tickers[index]["companyOfficers"]
            officers = sorted(officers, key = lambda x: x['unexercisedValue'], reverse = True)
            officer = officers[0]['name'].replace("Mr. ", "").replace("Ms. ", "").replace("Dr. ", "").replace("Mrs. ", "").replace(".", "").split(" ")
            officer = [x for x in officer if len(x) > 1]
            officer = " ".join(officer)
        except:
            officer = ""
        return officer

    def get_company_list(self):
        """Return a list of company information for later processing. Caches results to file."""
        if not os.path.exists('company_list.pkl'):
            self.yh_tickers = [Ticker(ticker) for ticker in self.stocks_symbols]
            start = timeit.default_timer()
            for t in range(len(self.yh_tickers)):
                sym = self.stocks_symbols[t]
                time_now = timeit.default_timer()
                print(f"{round(100*t/len(self.yh_tickers),1)}% complete.", "Estimated time left:", round((time_now-start)*(len(self.yh_tickers)-t)/(t+1),0), "seconds. Symbol:", self.stocks_symbols[t], end = '\r')
                self.yh_tickers[t] = self.yh_tickers[t].asset_profile[sym]
            with open('company_list.pkl', 'wb') as f:
                dump(self.yh_tickers, f)
        else:
            with open('company_list.pkl', 'rb') as f:
                self.yh_tickers = load(f)
    
    def process_information(self):
        self.data['Company'] = self.data['Ticker'].apply(self.get_company)
        self.data["Executive"] = [self.get_company_officers(t) for t in tqdm(range(len(self.yh_tickers)))] 

tickers = stock_tickers()
tickers.data.head()

  0%|          | 0/7105 [00:00<?, ?it/s]

,Ticker,Company,Executive
0,A,None,Michael McMullen
1,AA,None,Roy Harvey
2,AAC,None,David Kaplan
3,AACG,None,Xiaofeng Ma
4,AACI,None,Stephen Herbert


In [62]:
count = 0
for ticker in range(len(tickers.data)):
    if tickers.data['Ticker'][ticker] != "TSLA":
        count += 1
    else:
        print(tickers.data['Company'][ticker])
        print(tickers.data['Ticker'][ticker])
        print(ticker)
        break

None
TSLA
6436


In [70]:
officers = tickers.yh_tickers[6436]
officers

{'address1': '1 Tesla Road',
 'city': 'Austin',
 'state': 'TX',
 'zip': '78725',
 'country': 'United States',
 'phone': '512 516 8177',
 'website': 'https://www.tesla.com',
 'industry': 'Auto Manufacturers',
 'sector': 'Consumer Cyclical',
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. It operates in two segments, Automotive, and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits; and non-warranty after-sales vehicle, used vehicles, retail merchandise, and vehicle insurance services. This segment also provides sedans and sport utility vehicles through direct and used vehicle sales, a network of Tesla Superchargers, and in-app upgrades; purchase financing and leasing services; services for electric vehicles through its company-owned service locations and Tesla mo

In [64]:
# For every stock ticker symbol in stock_symbols, lookup the top executive and the company name.


Datasets:
We need a historical dataset of stock prices.
Secondly, we need a dataset of web trends for the same time period. 

On the other hand, once the model is trained, it needs to be able to aquire live data that looks the same as the training data.

Possible sources:
   - twitter
   - google trends

Possible models:
    - Train a model to process text into an encoding
    - Train a model to do stock price prediction
    - Combine the two models to predict stock price/volatility (not sure which is best) from text with greater accuracy

In [65]:
# Collect text web data

In [66]:
# Collect stock data

In [67]:
# 